In [1]:
import os
import pandas as pd
import re

In [2]:
def clean_column_name(col_name):
    return ''.join(e for e in col_name if e.isalnum()).upper()

def get_election_df(type):
    if type == 'AE':
        election_df = pd.read_csv('data/Chhattisgarh_AE_with_hindinames.csv')
    # if type == 'GA':
    #     election_df = pd.read_csv('data/Maharashtra_GA.csv')
        
    return election_df
        

In [4]:
def combine_files(year, excel_dir,output_dir, election_type):

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    ae_df = election_df[['Year','Candidate','Party']]
    ae_df = ae_df[ae_df['Year']==year]
    ae_df['Candidate'] = ae_df['Candidate'].apply(clean_column_name)

    mapping = dict(zip(ae_df['Candidate'], ae_df['Party']))

    # Get a list of all Excel files in the folder
    excel_files = [file for file in os.listdir(excel_dir) if file.endswith('.xlsx')]

    # Initialize an empty list to store DataFrames for each sheet
    dfs = []

    # Loop through each Excel file
    for file in excel_files:
        if '$' not in file:
            # Read all sheets from the Excel file into a dictionary of DataFrames
            t1 = pd.read_excel(os.path.join(excel_dir, file))#,skiprows=12,usecols='B,D,F,J')
            #t2 = pd.read_excel(os.path.join(folder_path, file), sheet_name='Table 2',skiprows=1)

            # Rename columns using the clean_column_name function
            #t1 = t1.rename(columns=clean_column_name)

            ac_number = int(file.split('.xlsx')[0])

            t1['AC_CODE'] = ac_number
            t1['PS_NO'] = t1['SN']

            # Rename columns in df2 using the mapping
            t1.rename(columns=mapping, inplace=True)

            
            dfs.append(t1)
            #dfs.append(t2)

    # Concatenate all DataFrames in the list into a single DataFrame
    concatenated_df = pd.concat(dfs, ignore_index=True)
    output_file_path = os.path.join(output_dir, f'combined_maharashtra_{year}_{election_type}.xlsx')
    concatenated_df.to_excel(output_file_path, index= False)

In [ ]:
# year = 2023 
year = 2019
# excel_dir = 'results/intermediate_tables/KA_2023'
excel_dir = 'results/intermediate_tables/CH/AE_2018'
# output_dir = 'results/strong_and_swing_booths/MH/combined_data/'
output_dir = 'results/MH/combined_data/'


election_type = 'AE' 
election_df = get_election_df(election_type)
combine_files(year, excel_dir,output_dir, election_type)


merge years

In [5]:
import pandas as pd

base_dir = 'results/MH/combined_data/'
s2=pd.read_excel(os.path.join(base_dir, 'combined_maharashtra_2019_AE.xlsx'))[['AC_CODE','PS_NO','INC_Status']]
s2.rename(columns={'INC_Status':'2019AEWin'},inplace=True)
s3=pd.read_excel(os.path.join(base_dir, 'combined_maharashtra_2019_GA.xlsx'))[['AC_CODE','PS_NO','INC_Status']]
s3.rename(columns={'INC_Status':'2019GAWin'},inplace=True)

# s1=pd.read_excel('2019 combined.xlsx')[['AC No.','Booth No.','2019Win']]

s12 = pd.merge(s2,s3,how='outer',on=['AC_CODE','PS_NO'])

s12.fillna('?',inplace=True)

output_file_path = os.path.join(base_dir, 'Booth Strength 2019_MH.xlsx')
s12.to_excel(output_file_path, index = False)

mergeboothsonaddressonly

In [6]:
import pandas as pd
import geopandas as gpd

booths = gpd.read_file('allBooths.geojson')
booths['AC_CODE'] = booths['ASBLY_CSTNY_NUM'].astype('str')
booths['PS_NO'] = booths['POLIN_STATN_NUM'].astype('str')

input_file_path = os.path.join('results/MH/combined_data/', 'Booth Strength 2019_MH.xlsx')
results = pd.read_excel(input_file_path)

results['AC_CODE'] = results['AC_CODE'].astype('str')
results['PS_NO'] = results['PS_NO'].astype('str')

merged = booths.merge(results,on=['AC_CODE','PS_NO'])

merged = merged[['AC_CODE','ASBLY_CSTNY_NAME', 'PS_NO', 'POLIN_STATN_NAME', '2019AEWin',
       '2019GAWin']]

output_file_path = os.path.join('results/MH/combined_data/', 'Booth Strength 2019_MH(w address).xlsx')

merged.to_excel(output_file_path,index=False)

In [ ]:
# Function to extract year from column names and determine booth strength
def determine_booth_strength(row, win_columns):
    wins = [row[col] for col in win_columns if col in row]
    
    if all(w == 'WON' for w in wins):
        return 'Strong'
    elif any(w == 'WON' for w in wins):
        return 'Swing'
    else:
        return None

In [ ]:
booth_strength_with_address_file_path = os.path.join('results/MHcombined_data/Booth Strength 2019_MH(w address).xlsx')
# Load your Excel file into a DataFrame
df = pd.read_excel(booth_strength_with_address_file_path)

# Get unique values from the "AC_Code" column
unique_ac_codes = df['AC_CODE'].unique()

os.makedirs('results/MH/combined_data/AC-wise', exist_ok=True)

win_columns = [col for col in df.columns if col.endswith('Win')]

# Loop through each unique AC_Code and create a separate Excel file
for ac_code in unique_ac_codes:
    # Filter the DataFrame for the current AC_Code
    filtered_df = df[df['AC_CODE'] == ac_code]

    filtered_df['Booth Strength'] = filtered_df.apply(determine_booth_strength, axis=1, win_columns=win_columns)

    # Create a new Excel file for the current AC_Code
    file_name = f'AC-wise/{ac_code}.xlsx'
    filtered_df.to_excel(os.path.join('results/MH/combined_data',file_name), index=False)

    filtered_df = filtered_df.drop_duplicates(subset=['AC_CODE','PS_NO'])

    print(f'Excel file "{file_name}" created successfully.')



In [7]:
import os
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Border, Side
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import CellIsRule
from openpyxl.styles import Alignment
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter

merge_range1 = 'A1:E1'
merge_range2 = 'A2:E2'
merge_range3 = 'A3:E3'


# Define the fill colors for conditional formatting
light_green_fill = PatternFill(start_color='90EE90', end_color='90EE90', fill_type='solid')
light_red_fill = PatternFill(start_color='FFA07A', end_color='FFA07A', fill_type='solid')

# Create conditional formatting rules
win_rule = CellIsRule(operator='equal', formula=['"Won"'], fill=light_green_fill)
loss_rule = CellIsRule(operator='equal', formula=['"Lost"'], fill=light_red_fill)

center_alignment = Alignment(horizontal='center',vertical='center')
# Path to the folder containing your Excel files
folder_path = "results/MH/combined_data/AC-wise"

# Get a list of all Excel files in the folder
excel_files = [file for file in os.listdir(folder_path) if file.endswith('.xlsx')]

cats = ['Strong','Swing']
border = Border(left=Side(style='thin'),
                right=Side(style='thin'),
                top=Side(style='thin'),
                bottom=Side(style='thin'))

# Loop through each Excel file
for file in excel_files:
    if '$' not in file:
        # Load the Excel file into a pandas DataFrame
        df = pd.read_excel(os.path.join(folder_path, file))
        AC_name = df['ASBLY_CSTNY_NAME'].unique()[0]
        for cat in cats:
            
            # Filter rows where column H contains the word "Swing"
            filtered_df = df[df['Booth Strength'].str.contains(cat, na=False)]

            column_rename_map = {
            '2019AEWin': '2019AE',
            '2019GAWin': '2019GA',
            'PS_NO':'PS No',
            'POLIN_STATN_NAME':'Booth Name'
            # Add more mappings as needed
            }

            # Rename columns using the rename() method
            filtered_df = filtered_df.rename(columns=column_rename_map)

            filtered_df = filtered_df[['PS No','Booth Name','2019AE','2019GA']]
            
            # Save the filtered DataFrame to a new Excel file
            output_file = file.replace('.xlsx','') + ' - '+cat+' only.xlsx'
            filtered_df.to_excel(os.path.join('results/MH/combined_data/strong-swing', output_file), index=False)

            # Load the Excel file
            wb = load_workbook(filename=os.path.join('results/MH/combined_data/strong-swing', output_file))
            # Choose the active worksheet
            ws = wb.active
            # Set the width of column A to 10 points
            ws.column_dimensions['A'].width = 5
            ws.column_dimensions['B'].width = 63
            ws.column_dimensions['C'].width = 5
            ws.column_dimensions['D'].width = 5
            ws.column_dimensions['E'].width = 5

            ws.conditional_formatting.add('A1:G300', win_rule)
            ws.conditional_formatting.add('A1:G300', loss_rule)


            for cell in ws[1]:
                cell.alignment = cell.alignment.copy(wrap_text=True)
                cell.alignment = cell.alignment.copy(horizontal='center',vertical='center')
            # Create a border style with all borders enabled

            # Apply the border style to all cells in the worksheet
            for row in ws.iter_rows():
                for cell in row:
                    cell.border = border

            for col in [1,3,4,5]:  # Columns C to G
                for cell in ws.iter_cols(min_col=col, max_col=col):
                    for c in cell:
                        c.alignment = center_alignment
            
            ws.insert_rows(1)
            ws.insert_rows(1)
            ws.insert_rows(1)
            ws.merge_cells(merge_range1)
            ws.merge_cells(merge_range2)
            ws.merge_cells(merge_range3)

            


            # Get the merged cell
            merged_cell1 = ws[merge_range1.split(":")[0]]
            # Set the cell value to "HEADING"
            merged_cell1.value = "Maharashtra Booth-wise Strength"
            merged_cell1.alignment = center_alignment
            ws.row_dimensions[1].height = 15

            # Get the merged cell
            merged_cell2 = ws[merge_range2.split(":")[0]]
            # Set the cell value to "HEADING"
            merged_cell2.value = "AC - "+AC_name+" ("+file.replace('.xlsx','')+')'
            merged_cell2.alignment = center_alignment
            ws.row_dimensions[2].height = 15

            # Get the merged cell
            merged_cell3 = ws[merge_range3.split(":")[0]]
            # Set the cell value to "HEADING"
            merged_cell3.value = cat+" Booths"
            merged_cell3.alignment = center_alignment
            ws.row_dimensions[3].height = 15

            ws.row_dimensions[4].height = 20

            # Define the bold font style
            bold_font = Font(bold=True)

            # Apply bold font style to the merged cell
            merged_cell3.font = bold_font
            merged_cell2.font = bold_font
            merged_cell1.font = bold_font

            output_file_path = os.path.join('results/MH/combined_data/strong-swing', output_file)
            # Save the workbook to a file
            wb.save(output_file_path)